# 01_CAE Pipeline — Model Training and Evaluation

## Purpose
This notebook implements the full training and evaluation pipeline for a **Convolutional Autoencoder (CAE)** designed to reconstruct 2D Navier–Stokes flow fields. The CAE serves as a baseline deep-learning model, learning a compressed latent representation that preserves the essential physical structures of the fluid flow—such as vortices, temperature gradients, and pressure variations. The model is trained on flows with Reynolds numbers up to 1000 and evaluated on both in-distribution (“normal”) and out-of-distribution (“hard”) flow regimes to assess generalization performance.

---

## Contents
This notebook includes the following components:

- **Data preprocessing**
  - Loading `.npy` simulation files  
  - Removing the vertical velocity channel (*v*), which contains no useful flow information  
  - Applying per-channel standardization using training-set statistics  

- **Model architecture**
  - Encoder with strided convolutions  
  - Latent bottleneck representation  
  - Decoder using transposed convolutions  

- **Training loop (200 epochs)**
  - MSE loss  
  - Adam optimizer  
  - Batch-wise training and validation  

- **Evaluation**
  - Metrics: **MSE** and **PSNR**
  - Evaluation on Train, Validation, Test-Normal, and Test-Hard  

- **Outputs**
  - Training-curve plots  
  - Reconstruction examples  
  - Saved metrics  

---


## Pipeline Overview

### 1. Data Loading and Preprocessing
The notebook loads `.npy` files containing 4-channel fields:  
`[u, v, T, p]`.

From exploratory analysis, the **v channel was shown to contain almost no variance**, providing no useful signal for learning fluid dynamics. Therefore, this channel is removed, reducing the inputs to:

- Horizontal velocity (*u*)
- Temperature (*T*)
- Pressure (*p*)

The notebook then computes **per-channel mean and standard deviation on the training set**, applying standardization to all data splits. This ensures consistent scaling across the dataset and prevents data leakage.

---

In [ ]:
#Imports, Device Setup, Paths

import os
import glob
import re
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# My folders. Chnage the paths if you are trying to reproduce the pipeline.

train_folder = "../DATA/NavierStokes/test"
test_folder  = "../DATA/NavierStokes/test"

# Use up to Re ≤ 1000 for training-range flows
MAX_RE_TRAIN = 1000


In [ ]:
# Loader Functions (for train and test folders)

def load_ns_folder_max_re(folder_path, max_re=None):
    """
    Loads all .npy simulation files whose Reynolds number <= max_re.
    Drops channel index 1 (vertical velocity), keeps 0, 2, 3.
    Returns: numpy array (N, 3, H, W).
    """
    all_arrays = []
    npy_files = sorted(glob.glob(os.path.join(folder_path, "Re_*.npy")))
    print(f"[TRAIN LOADER] Found {len(npy_files)} files in {folder_path}")

    selected_files = []
    for fpath in npy_files:
        fname = os.path.basename(fpath)
        m = re.match(r"Re_(\d+)\.npy", fname)
        if not m:
            continue
        re_val = int(m.group(1))
        if max_re is not None and re_val > max_re:
            continue
        selected_files.append((re_val, fpath))

    if not selected_files:
        raise RuntimeError("No train files selected — check filters.")

    print("[TRAIN LOADER] Using files:")
    for re_val, fpath in selected_files:
        print(f"  Re={re_val:<5} -> {os.path.basename(fpath)}")

    for re_val, fpath in selected_files:
        data = np.load(fpath)  # expected shape (T, 4, H, W)
        data_3ch = np.stack([data[:,0], data[:,2], data[:,3]], axis=1)
        all_arrays.append(data_3ch)

    all_data = np.concatenate(all_arrays, axis=0)
    print("[TRAIN LOADER] Combined shape:", all_data.shape)
    return all_data


def load_ns_folder_test_split(folder_path):
    """
    Splits test folder into:
    - test_normal: Re ≤ 1000
    - test_hard:   Re ≥ 2000
    Drops channel index 1.
    """
    all_normal = []
    all_hard   = []

    npy_files = sorted(glob.glob(os.path.join(folder_path, "Re_*.npy")))
    print(f"[TEST LOADER] Found {len(npy_files)} files in {folder_path}")

    for fpath in npy_files:
        fname = os.path.basename(fpath)
        m = re.match(r"Re_(\d+)\.npy", fname)
        if not m:
            continue
        re_val = int(m.group(1))

        data = np.load(fpath)
        data_3ch = np.stack([data[:,0], data[:,2], data[:,3]], axis=1)

        if re_val <= 1000:
            all_normal.append(data_3ch)
        else:
            all_hard.append(data_3ch)

    test_normal = np.concatenate(all_normal, axis=0)
    print("[TEST LOADER] test_normal shape:", test_normal.shape)

    if len(all_hard) > 0:
        test_hard = np.concatenate(all_hard, axis=0)
        print("[TEST LOADER] test_hard   shape:", test_hard.shape)
    else:
        test_hard = None
        print("[TEST LOADER] No hard test files found.")

    return test_normal, test_hard


In [ ]:
# Load train data (Re ≤ 1000) and test split

train_data = load_ns_folder_max_re(train_folder, max_re=MAX_RE_TRAIN)
test_normal, test_hard = load_ns_folder_test_split(test_folder)

print("Train data shape:", train_data.shape)
print("Test normal shape:", test_normal.shape)
print("Test hard shape:", None if test_hard is None else test_hard.shape)


In [ ]:
# Per-channel normalization using train stats

def compute_channel_stats(x):
    mean = x.mean(axis=(0,2,3))
    std  = x.std(axis=(0,2,3))
    return mean, std

train_mean, train_std = compute_channel_stats(train_data)
train_std_safe = np.where(train_std < 1e-8, 1.0, train_std)

print("Train mean:", train_mean)
print("Train std :", train_std_safe)

def normalize(x, mean, std):
    x_norm = (x - mean[None,:,None,None]) / std[None,:,None,None]
    x_norm = np.clip(x_norm, -3.0, 3.0)
    return x_norm

train_data_norm = normalize(train_data, train_mean, train_std_safe)
test_normal_norm = normalize(test_normal, train_mean, train_std_safe)
test_hard_norm = normalize(test_hard, train_mean, train_std_safe) if test_hard is not None else None

np.savez("../OUTPUTS/01_CAE_pipeline_outputs/ns_norm_stats.npz", mean=train_mean, std=train_std_safe)

print("Normalized shapes:")
print(" train:", train_data_norm.shape)
print(" test_normal:", test_normal_norm.shape)
print(" test_hard:", None if test_hard_norm is None else test_hard_norm.shape)


In [ ]:
# Dataset and DataLoaders

class NSDataset(Dataset):
    def __init__(self, data_array):
        self.data = data_array.astype(np.float32)

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return torch.from_numpy(self.data[idx])

# Create datasets
full_train_dataset = NSDataset(train_data_norm)
test_normal_dataset = NSDataset(test_normal_norm)
test_hard_dataset = NSDataset(test_hard_norm) if test_hard_norm is not None else None

# Train/Val split
val_ratio = 0.1
val_size = int(len(full_train_dataset) * val_ratio)
train_size = len(full_train_dataset) - val_size

train_dataset, val_dataset = torch.utils.data.random_split(
    full_train_dataset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

batch_size = 8  # safe for CPU

train_loader       = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader         = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)
test_normal_loader = DataLoader(test_normal_dataset, batch_size=batch_size, shuffle=False)
test_hard_loader   = DataLoader(test_hard_dataset,   batch_size=batch_size, shuffle=False) if test_hard_dataset else None

print("Train/Val/Test sizes:", len(train_dataset), len(val_dataset), len(test_normal_dataset))


### 2. CAE Architecture
The CAE follows a symmetric encoder–decoder structure:

- **Encoder:** progressively downsamples the 3-channel input using convolutional layers with stride-2, extracting hierarchical spatial features.
- **Latent space:** captures the global structure of the flow fields in a highly compressed form.
- **Decoder:** reconstructs the full spatial resolution using transposed convolutions.

This architecture is commonly used for image-like scientific data and is well-suited for reconstructing complex fluid structures.

---

In [ ]:
#Standard/Simple CAE pipeline

class SimpleCAE(nn.Module):
    def __init__(self, in_channels=3, base_channels=32):
        super().__init__()

        self.enc1 = nn.Sequential(
            nn.Conv2d(in_channels, base_channels, 3, padding=1),
            nn.BatchNorm2d(base_channels),
            nn.ReLU(True),
        )
        self.enc2 = nn.Sequential(
            nn.Conv2d(base_channels, base_channels*2, 3, stride=2, padding=1),
            nn.BatchNorm2d(base_channels*2),
            nn.ReLU(True),
        )
        self.enc3 = nn.Sequential(
            nn.Conv2d(base_channels*2, base_channels*4, 3, stride=2, padding=1),
            nn.BatchNorm2d(base_channels*4),
            nn.ReLU(True),
        )
        self.enc4 = nn.Sequential(
            nn.Conv2d(base_channels*4, base_channels*8, 3, stride=2, padding=1),
            nn.BatchNorm2d(base_channels*8),
            nn.ReLU(True),
        )

        self.dec1 = nn.Sequential(
            nn.ConvTranspose2d(base_channels*8, base_channels*4, 4, stride=2, padding=1),
            nn.BatchNorm2d(base_channels*4),
            nn.ReLU(True),
        )
        self.dec2 = nn.Sequential(
            nn.ConvTranspose2d(base_channels*4, base_channels*2, 4, stride=2, padding=1),
            nn.BatchNorm2d(base_channels*2),
            nn.ReLU(True),
        )
        self.dec3 = nn.Sequential(
            nn.ConvTranspose2d(base_channels*2, base_channels, 4, stride=2, padding=1),
            nn.BatchNorm2d(base_channels),
            nn.ReLU(True),
        )

        self.final = nn.Conv2d(base_channels, in_channels, 3, padding=1)

    def forward(self, x):
        x = self.enc1(x)
        x = self.enc2(x)
        x = self.enc3(x)
        x = self.enc4(x)
        x = self.dec1(x)
        x = self.dec2(x)
        x = self.dec3(x)
        x = self.final(x)
        return x

model = SimpleCAE(in_channels=3, base_channels=32).to(device)
model


### 3. Training Loop (200 Epochs)
The CAE is trained for 200 epochs using:

- **Loss:** Mean Squared Error (MSE)  
- **Optimizer:** Adam  
- **Batch Size:** 16  
- **Device:** CPU  

The training curve shows rapid loss reduction early on, followed by stable convergence to low reconstruction error. Validation loss closely tracks training loss, indicating good generalization.

---

In [ ]:
# Train CAE

criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

num_epochs = 200  # Start small, increase to 30–40 if fast

for epoch in range(1, num_epochs + 1):
    # Training 
    model.train()
    train_loss = 0.0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        outputs = model(batch)
        loss = criterion(outputs, batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * batch.size(0)
    train_loss /= len(train_loader.dataset)

    # Validation 
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            batch = batch.to(device)
            outputs = model(batch)
            loss = criterion(outputs, batch)
            val_loss += loss.item() * batch.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {epoch:03d} | Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f}")


### 4.  CAE Final Evaluation Metrics

| Dataset Split   | MSE        | PSNR (dB) |
|-----------------|------------|-----------|
| Train           | 0.001362   | 44.22     |
| Validation      | 0.001340   | 44.29     |
| Test (Normal)   | 0.001994   | 42.57     |
| Test (Hard)     | 0.028275   | 31.05     |

**Summary:**  
- The CAE achieves strong reconstruction performance on training and validation datasets, with PSNR values above 44 dB.  
- Performance remains high for normal test flows (Re ≤ 1000).  
- The model experiences expected degradation on high-Re out-of-distribution flows, with PSNR dropping to ~31 dB in the Test-Hard regime.  

---


In [ ]:
# Evaluation Helpers

def eval_mse_psnr(model, loader, name="set"):
    model.eval()
    mse_sum = 0.0
    n = 0
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            output = model(batch)
            mse = criterion(output, batch).item()
            mse_sum += mse * batch.size(0)
            n += batch.size(0)

    avg_mse = mse_sum / n
    max_val = 6.0  # normalized range approximately [-3, 3]
    psnr = 10 * np.log10((max_val ** 2) / avg_mse) if avg_mse > 0 else float("inf")

    print(f"{name}: MSE={avg_mse:.6f}, PSNR={psnr:.2f} dB")
    return avg_mse, psnr


# Evaluate on all sets
eval_mse_psnr(model, train_loader,       "Train")
eval_mse_psnr(model, val_loader,         "Val")
eval_mse_psnr(model, test_normal_loader, "Test (normal)")

if test_hard_loader:
    eval_mse_psnr(model, test_hard_loader, "Test (hard)")


### 5. Outputs
This notebook generates the following outputs:

- **`cae_training_curve.png`** — the full 200-epoch training/validation loss curve  
- **Reconstruction examples** for several Reynolds numbers (original vs CAE output)  
- **`cae_metrics.json`** containing final MSE and PSNR values for all dataset splits  
- **MP4 reconstruction videos** (optional, produced if video export is enabled)

These outputs serve as the baseline results used to compare the CAE against the DAE model.


In [ ]:
# Denormalization helper

def denormalize(x_norm, mean, std):
    """
    x_norm: tensor (B, 3, H, W)
    mean/std: numpy arrays of shape (3,)
    returns: tensor in original unnormalized range
    """
    mean_t = torch.tensor(mean, dtype=torch.float32, device=x_norm.device)[None, :, None, None]
    std_t  = torch.tensor(std,  dtype=torch.float32, device=x_norm.device)[None, :, None, None]
    return x_norm * std_t + mean_t


In [ ]:
# Visualization function

import matplotlib.pyplot as plt

def visualize_reconstruction(model, loader, mean, std, num_samples=1):
    model.eval()
    batch = next(iter(loader))
    batch = batch.to(device)

    with torch.no_grad():
        recon = model(batch)

    # denormalize
    orig_dn  = denormalize(batch, mean, std).cpu().numpy()
    recon_dn = denormalize(recon, mean, std).cpu().numpy()

    for i in range(num_samples):
        fig, axs = plt.subplots(3, 3, figsize=(12, 10))
        fig.suptitle(f"Sample {i}", fontsize=16)

        for ch, title in enumerate(["u-velocity", "Temperature", "Pressure"]):
            orig_img = orig_dn[i, ch]
            recon_img = recon_dn[i, ch]
            error_img = np.abs(orig_img - recon_img)

            # Original
            axs[ch, 0].imshow(orig_img, cmap="viridis")
            axs[ch, 0].set_title(f"{title} (Original)")
            axs[ch, 0].axis("off")

            # Reconstruction
            axs[ch, 1].imshow(recon_img, cmap="viridis")
            axs[ch, 1].set_title(f"{title} (Reconstruction)")
            axs[ch, 1].axis("off")

            # Error
            axs[ch, 2].imshow(error_img, cmap="inferno")
            axs[ch, 2].set_title(f"{title} (Error)")
            axs[ch, 2].axis("off")

        plt.tight_layout()
        plt.show()


In [ ]:
# Visualize reconstruction on normal test flows
visualize_reconstruction(
    model,
    test_normal_loader,
    mean=train_mean,
    std=train_std_safe,
    num_samples=2     # increase for more examples
)


In [ ]:
# Visualize reconstruction on hard test flows
if test_hard_loader is not None:
    visualize_reconstruction(
        model,
        test_hard_loader,
        mean=train_mean,
        std=train_std_safe,
        num_samples=2     # increase for more
    )
else:
    print("No hard test flows available.")


In [ ]:
# Choose a clear, fixed output directory for videos
video_output_dir = "../OUTPUTS/01_CAE_pipeline_outputs/videos"
print("Videos will be saved to:", video_output_dir)


In [ ]:
#The videos mentioned above will be attach to our repository on Github.

In [ ]:
import re
import matplotlib.pyplot as plt

log_text = """
Epoch 001 | Train Loss: 0.131040 | Val Loss: 0.034471
Epoch 002 | Train Loss: 0.048356 | Val Loss: 0.035027
Epoch 003 | Train Loss: 0.033534 | Val Loss: 0.025850
Epoch 004 | Train Loss: 0.024444 | Val Loss: 0.009369
Epoch 005 | Train Loss: 0.025980 | Val Loss: 0.023159
Epoch 006 | Train Loss: 0.017462 | Val Loss: 0.009678
Epoch 007 | Train Loss: 0.016250 | Val Loss: 0.010336
Epoch 008 | Train Loss: 0.020353 | Val Loss: 0.009575
Epoch 009 | Train Loss: 0.013735 | Val Loss: 0.014295
Epoch 010 | Train Loss: 0.019471 | Val Loss: 0.012905
Epoch 011 | Train Loss: 0.013030 | Val Loss: 0.013214
Epoch 012 | Train Loss: 0.013155 | Val Loss: 0.008594
Epoch 013 | Train Loss: 0.011065 | Val Loss: 0.008925
Epoch 014 | Train Loss: 0.013727 | Val Loss: 0.008770
Epoch 015 | Train Loss: 0.009819 | Val Loss: 0.005979
Epoch 016 | Train Loss: 0.009272 | Val Loss: 0.006265
Epoch 017 | Train Loss: 0.011339 | Val Loss: 0.008362
Epoch 018 | Train Loss: 0.010298 | Val Loss: 0.003655
Epoch 019 | Train Loss: 0.011214 | Val Loss: 0.005608
Epoch 020 | Train Loss: 0.007265 | Val Loss: 0.010508
Epoch 021 | Train Loss: 0.009443 | Val Loss: 0.004416
Epoch 022 | Train Loss: 0.007108 | Val Loss: 0.003338
Epoch 023 | Train Loss: 0.008502 | Val Loss: 0.004409
Epoch 024 | Train Loss: 0.007876 | Val Loss: 0.004467
Epoch 025 | Train Loss: 0.006847 | Val Loss: 0.007904
Epoch 026 | Train Loss: 0.009264 | Val Loss: 0.005870
Epoch 027 | Train Loss: 0.006773 | Val Loss: 0.004234
Epoch 028 | Train Loss: 0.008599 | Val Loss: 0.016466
Epoch 029 | Train Loss: 0.007612 | Val Loss: 0.005221
Epoch 030 | Train Loss: 0.005708 | Val Loss: 0.003803
Epoch 031 | Train Loss: 0.006048 | Val Loss: 0.002006
Epoch 032 | Train Loss: 0.005417 | Val Loss: 0.005125
Epoch 033 | Train Loss: 0.006553 | Val Loss: 0.004365
Epoch 034 | Train Loss: 0.005932 | Val Loss: 0.007277
Epoch 035 | Train Loss: 0.007295 | Val Loss: 0.004203
Epoch 036 | Train Loss: 0.006642 | Val Loss: 0.004723
Epoch 037 | Train Loss: 0.009999 | Val Loss: 0.015419
Epoch 038 | Train Loss: 0.007696 | Val Loss: 0.013741
Epoch 039 | Train Loss: 0.011325 | Val Loss: 0.003242
Epoch 040 | Train Loss: 0.006824 | Val Loss: 0.004654
Epoch 041 | Train Loss: 0.004719 | Val Loss: 0.004446
Epoch 042 | Train Loss: 0.006647 | Val Loss: 0.002584
Epoch 043 | Train Loss: 0.006190 | Val Loss: 0.004814
Epoch 044 | Train Loss: 0.005997 | Val Loss: 0.007286
Epoch 045 | Train Loss: 0.004629 | Val Loss: 0.008094
Epoch 046 | Train Loss: 0.005457 | Val Loss: 0.005376
Epoch 047 | Train Loss: 0.005300 | Val Loss: 0.003762
Epoch 048 | Train Loss: 0.004720 | Val Loss: 0.004346
Epoch 049 | Train Loss: 0.006235 | Val Loss: 0.004375
Epoch 050 | Train Loss: 0.005613 | Val Loss: 0.003302
Epoch 051 | Train Loss: 0.004942 | Val Loss: 0.003055
Epoch 052 | Train Loss: 0.004975 | Val Loss: 0.003715
Epoch 053 | Train Loss: 0.004905 | Val Loss: 0.002990
Epoch 054 | Train Loss: 0.005766 | Val Loss: 0.009009
Epoch 055 | Train Loss: 0.006048 | Val Loss: 0.003627
Epoch 056 | Train Loss: 0.004465 | Val Loss: 0.003209
Epoch 057 | Train Loss: 0.005965 | Val Loss: 0.013057
Epoch 058 | Train Loss: 0.024332 | Val Loss: 0.011620
Epoch 059 | Train Loss: 0.006425 | Val Loss: 0.003395
Epoch 060 | Train Loss: 0.006148 | Val Loss: 0.002262
Epoch 061 | Train Loss: 0.004509 | Val Loss: 0.004793
Epoch 062 | Train Loss: 0.004621 | Val Loss: 0.005715
Epoch 063 | Train Loss: 0.004872 | Val Loss: 0.005605
Epoch 064 | Train Loss: 0.004870 | Val Loss: 0.003204
Epoch 065 | Train Loss: 0.007419 | Val Loss: 0.002560
Epoch 066 | Train Loss: 0.004479 | Val Loss: 0.004866
Epoch 067 | Train Loss: 0.004821 | Val Loss: 0.003335
Epoch 068 | Train Loss: 0.004827 | Val Loss: 0.004255
Epoch 069 | Train Loss: 0.005003 | Val Loss: 0.004187
Epoch 070 | Train Loss: 0.004473 | Val Loss: 0.003609
Epoch 071 | Train Loss: 0.003957 | Val Loss: 0.002782
Epoch 072 | Train Loss: 0.006179 | Val Loss: 0.003294
Epoch 073 | Train Loss: 0.005109 | Val Loss: 0.002542
Epoch 074 | Train Loss: 0.003372 | Val Loss: 0.003930
Epoch 075 | Train Loss: 0.004483 | Val Loss: 0.002552
Epoch 076 | Train Loss: 0.003404 | Val Loss: 0.003570
Epoch 077 | Train Loss: 0.003150 | Val Loss: 0.002404
Epoch 078 | Train Loss: 0.002954 | Val Loss: 0.001977
Epoch 079 | Train Loss: 0.003374 | Val Loss: 0.003172
Epoch 080 | Train Loss: 0.003096 | Val Loss: 0.003690
Epoch 081 | Train Loss: 0.003595 | Val Loss: 0.002123
Epoch 082 | Train Loss: 0.003660 | Val Loss: 0.007555
Epoch 083 | Train Loss: 0.012364 | Val Loss: 0.003798
Epoch 084 | Train Loss: 0.002962 | Val Loss: 0.002520
Epoch 085 | Train Loss: 0.003675 | Val Loss: 0.001890
Epoch 086 | Train Loss: 0.002989 | Val Loss: 0.003051
Epoch 087 | Train Loss: 0.004096 | Val Loss: 0.003176
Epoch 088 | Train Loss: 0.003420 | Val Loss: 0.002202
Epoch 089 | Train Loss: 0.003668 | Val Loss: 0.001972
Epoch 090 | Train Loss: 0.003292 | Val Loss: 0.001889
Epoch 091 | Train Loss: 0.003124 | Val Loss: 0.002094
Epoch 092 | Train Loss: 0.003269 | Val Loss: 0.002542
Epoch 093 | Train Loss: 0.003514 | Val Loss: 0.003164
Epoch 094 | Train Loss: 0.002930 | Val Loss: 0.002304
Epoch 095 | Train Loss: 0.002728 | Val Loss: 0.004481
Epoch 096 | Train Loss: 0.003069 | Val Loss: 0.001529
Epoch 097 | Train Loss: 0.002868 | Val Loss: 0.002272
Epoch 098 | Train Loss: 0.003397 | Val Loss: 0.003904
Epoch 099 | Train Loss: 0.003374 | Val Loss: 0.002752
Epoch 100 | Train Loss: 0.003318 | Val Loss: 0.002839
Epoch 101 | Train Loss: 0.002545 | Val Loss: 0.006645
Epoch 102 | Train Loss: 0.003221 | Val Loss: 0.001527
Epoch 103 | Train Loss: 0.002604 | Val Loss: 0.003190
Epoch 104 | Train Loss: 0.002668 | Val Loss: 0.001990
Epoch 105 | Train Loss: 0.003163 | Val Loss: 0.002748
Epoch 106 | Train Loss: 0.002708 | Val Loss: 0.003297
Epoch 107 | Train Loss: 0.003405 | Val Loss: 0.003402
Epoch 108 | Train Loss: 0.002956 | Val Loss: 0.002359
Epoch 109 | Train Loss: 0.003498 | Val Loss: 0.002795
Epoch 110 | Train Loss: 0.002998 | Val Loss: 0.002097
Epoch 111 | Train Loss: 0.003293 | Val Loss: 0.003470
Epoch 112 | Train Loss: 0.002430 | Val Loss: 0.003261
Epoch 113 | Train Loss: 0.006032 | Val Loss: 0.002359
Epoch 114 | Train Loss: 0.002420 | Val Loss: 0.002534
Epoch 115 | Train Loss: 0.004905 | Val Loss: 0.014279
Epoch 116 | Train Loss: 0.012050 | Val Loss: 0.004581
Epoch 117 | Train Loss: 0.003617 | Val Loss: 0.001642
Epoch 118 | Train Loss: 0.002423 | Val Loss: 0.002267
Epoch 119 | Train Loss: 0.003152 | Val Loss: 0.002573
Epoch 120 | Train Loss: 0.002093 | Val Loss: 0.001280
Epoch 121 | Train Loss: 0.003534 | Val Loss: 0.001297
Epoch 122 | Train Loss: 0.001943 | Val Loss: 0.002293
Epoch 123 | Train Loss: 0.003057 | Val Loss: 0.003165
Epoch 124 | Train Loss: 0.003576 | Val Loss: 0.001475
Epoch 125 | Train Loss: 0.002805 | Val Loss: 0.008155
Epoch 126 | Train Loss: 0.003250 | Val Loss: 0.001314
Epoch 127 | Train Loss: 0.002036 | Val Loss: 0.001446
Epoch 128 | Train Loss: 0.002127 | Val Loss: 0.001795
Epoch 129 | Train Loss: 0.002098 | Val Loss: 0.001325
Epoch 130 | Train Loss: 0.001786 | Val Loss: 0.001616
Epoch 131 | Train Loss: 0.002358 | Val Loss: 0.002072
Epoch 132 | Train Loss: 0.002266 | Val Loss: 0.003145
Epoch 133 | Train Loss: 0.002941 | Val Loss: 0.003448
Epoch 134 | Train Loss: 0.002163 | Val Loss: 0.001917
Epoch 135 | Train Loss: 0.003559 | Val Loss: 0.004786
Epoch 136 | Train Loss: 0.002548 | Val Loss: 0.003181
Epoch 137 | Train Loss: 0.003393 | Val Loss: 0.001452
Epoch 138 | Train Loss: 0.002757 | Val Loss: 0.004278
Epoch 139 | Train Loss: 0.002541 | Val Loss: 0.002029
Epoch 140 | Train Loss: 0.002337 | Val Loss: 0.001636
Epoch 141 | Train Loss: 0.001568 | Val Loss: 0.002757
Epoch 142 | Train Loss: 0.002057 | Val Loss: 0.003430
Epoch 143 | Train Loss: 0.002120 | Val Loss: 0.001094
Epoch 144 | Train Loss: 0.001681 | Val Loss: 0.000901
Epoch 145 | Train Loss: 0.002024 | Val Loss: 0.002382
Epoch 146 | Train Loss: 0.002472 | Val Loss: 0.002531
Epoch 147 | Train Loss: 0.002296 | Val Loss: 0.001311
Epoch 148 | Train Loss: 0.002236 | Val Loss: 0.001599
Epoch 149 | Train Loss: 0.001817 | Val Loss: 0.002535
Epoch 150 | Train Loss: 0.002725 | Val Loss: 0.001850
Epoch 151 | Train Loss: 0.002303 | Val Loss: 0.002685
Epoch 152 | Train Loss: 0.002384 | Val Loss: 0.001978
Epoch 153 | Train Loss: 0.002269 | Val Loss: 0.001225
Epoch 154 | Train Loss: 0.001779 | Val Loss: 0.003180
Epoch 155 | Train Loss: 0.001545 | Val Loss: 0.002364
Epoch 156 | Train Loss: 0.002348 | Val Loss: 0.002853
Epoch 157 | Train Loss: 0.002939 | Val Loss: 0.001634
Epoch 158 | Train Loss: 0.002908 | Val Loss: 0.002000
Epoch 159 | Train Loss: 0.001868 | Val Loss: 0.002100
Epoch 160 | Train Loss: 0.001986 | Val Loss: 0.002208
Epoch 161 | Train Loss: 0.001746 | Val Loss: 0.001383
Epoch 162 | Train Loss: 0.001813 | Val Loss: 0.001342
Epoch 163 | Train Loss: 0.001883 | Val Loss: 0.001286
Epoch 164 | Train Loss: 0.001449 | Val Loss: 0.000860
Epoch 165 | Train Loss: 0.002079 | Val Loss: 0.001555
Epoch 166 | Train Loss: 0.002310 | Val Loss: 0.001461
Epoch 167 | Train Loss: 0.002912 | Val Loss: 0.001416
Epoch 168 | Train Loss: 0.002502 | Val Loss: 0.002016
Epoch 169 | Train Loss: 0.003415 | Val Loss: 0.003496
Epoch 170 | Train Loss: 0.001627 | Val Loss: 0.001453
Epoch 171 | Train Loss: 0.001661 | Val Loss: 0.001818
Epoch 172 | Train Loss: 0.001388 | Val Loss: 0.001380
Epoch 173 | Train Loss: 0.001480 | Val Loss: 0.002925
Epoch 174 | Train Loss: 0.001651 | Val Loss: 0.000788
Epoch 175 | Train Loss: 0.001914 | Val Loss: 0.002806
Epoch 176 | Train Loss: 0.004301 | Val Loss: 0.001652
Epoch 177 | Train Loss: 0.001915 | Val Loss: 0.001387
Epoch 178 | Train Loss: 0.001624 | Val Loss: 0.001559
Epoch 179 | Train Loss: 0.002091 | Val Loss: 0.001515
Epoch 180 | Train Loss: 0.002194 | Val Loss: 0.001295
Epoch 181 | Train Loss: 0.002850 | Val Loss: 0.001799
Epoch 182 | Train Loss: 0.002056 | Val Loss: 0.003284
Epoch 183 | Train Loss: 0.001490 | Val Loss: 0.002176
Epoch 184 | Train Loss: 0.001588 | Val Loss: 0.001726
Epoch 185 | Train Loss: 0.002845 | Val Loss: 0.001494
Epoch 186 | Train Loss: 0.002129 | Val Loss: 0.001819
Epoch 187 | Train Loss: 0.001779 | Val Loss: 0.001181
Epoch 188 | Train Loss: 0.001602 | Val Loss: 0.001886
Epoch 189 | Train Loss: 0.001305 | Val Loss: 0.002187
Epoch 190 | Train Loss: 0.001538 | Val Loss: 0.001800
Epoch 191 | Train Loss: 0.002182 | Val Loss: 0.001184
Epoch 192 | Train Loss: 0.001757 | Val Loss: 0.003271
Epoch 193 | Train Loss: 0.001330 | Val Loss: 0.001495
Epoch 194 | Train Loss: 0.001390 | Val Loss: 0.002466
Epoch 195 | Train Loss: 0.001443 | Val Loss: 0.001360
Epoch 196 | Train Loss: 0.001335 | Val Loss: 0.000796
Epoch 197 | Train Loss: 0.001374 | Val Loss: 0.001150
Epoch 198 | Train Loss: 0.001416 | Val Loss: 0.001098
Epoch 199 | Train Loss: 0.001723 | Val Loss: 0.001630
Epoch 200 | Train Loss: 0.001700 | Val Loss: 0.001340
"""

# Parse using regex
train_losses = []
val_losses = []

for line in log_text.strip().splitlines():
    m = re.search(r"Train Loss:\s*([0-9.]+)\s*\|\s*Val Loss:\s*([0-9.]+)", line)
    if m:
        train_losses.append(float(m.group(1)))
        val_losses.append(float(m.group(2)))

print("Parsed epochs:", len(train_losses), "train losses,", len(val_losses), "val losses")


In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(train_losses, label="Train loss")
plt.plot(val_losses, label="Val loss")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.title("CAE Training Curve (200 epochs)")
plt.grid(linestyle="--", alpha=0.5)
plt.legend()
plt.tight_layout()
plt.show()
